# Find what's known for a list of genes

> eg. Maria L in the fly group routinely screens lists of human genes for the presence of evidence in neurological diseases. She wants to understand a bit about the biology of these genes. She looks at open targets to find what is known about the target and to which disease it has already been connected.

Compiling evidence relative to a list of genes can be a time-consuming and tedious task. Now the process can be automated, thanks to the targetvalidation.org API.

This tutorial illustrates how to compile all the evidence contained in targetvalidation.org for a list of genes into a compact representation (eg. a spreadsheet) using python. 
You might have a list of genes being generated from an experiment or a genetic screen, or perhaps you are following a particular set of targets and would like to maintain an updated look at what evidence has been compiled around them.
The process can be useful to reduce bias and increase success rates when prioritizing a list of genes for experimental follow-up. 

For this tutorial, we will use 20 random genes taken from the HGNC catalog. First, let's download the full catalog and create a list of symbols only.

In [2]:
%%bash
[ -f hgnc_complete_set.txt.gz ] || wget http://ftp.ebi.ac.uk/pub/databases/genenames/hgnc_complete_set.txt.gz 
[ -f hgnc_complete_set.txt ] || gunzip hgnc_complete_set.txt.gz 
tail -n +2 hgnc_complete_set.txt | cut -f2 | grep -v withdrawn > hgnc_symbols.txt

gunzip: hgnc_complete_set.txt already exists -- skipping
cut: stdin: Illegal byte sequence


To extract 20 random gene names:

In [1]:
import random
with open('hgnc_symbols.txt') as f:
    genes = [line.rstrip() for line in f]
random.shuffle(genes)
genes[:20]

['CACYBP',
 'CST4',
 'DUXA',
 'EIF4A1P10',
 'C6orf201',
 'ECSIT',
 'CC2D1B',
 'CFAP161',
 'FTO',
 'BCL2L15',
 'CDC27P3',
 'FREM1',
 'GOLIM4',
 'FER1L6-AS2',
 'ADCK5',
 'FAM168A',
 'COX6C',
 'C17orf96',
 'FAM151A',
 'AOC4P']

In [ ]:
#how many disease in each therapeutic area are associated with it

In [9]:
import requests
API = 'https://www.targetvalidation.org/api/1.1'

In [16]:
r = requests.post(API + '/public/association/filter', json = {'target':['ENSG00000157764','ENSG00000142168']})

In [18]:
r.url

'https://www.targetvalidation.org/api/1.1/public/association/filter'